In [1]:
!pip install langchain-openai==0.1.7
!pip install langchain==0.2.1
!pip install langchainhub==0.1.16
!pip install langchain-community==0.2.1
!pip install langchain-text-splitters==0.2.0

In [6]:
!pip install langchain-huggingface

In [7]:
!pip install pypdf==4.2.0
!pip install InstructorEmbedding==1.0.1
#!pip install sentence-transformers==2.6.0
!pip install faiss-gpu==1.7.2

# Tools

## PDF Retreival Tool

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader

In [9]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


embedding_model = HuggingFaceEmbeddings(
                    model_name="mixedbread-ai/mxbai-embed-large-v1",
            )

#texts = ["Hello, world!", "How are you?"]
#query_result = embedding_model.embed_documents(texts)

In [10]:
!wget --no-clobber https://raw.githubusercontent.com/DoranLyong/langchain-tutorial-kor/main/Agent/data/SPRI_AI_Brief_2024_May.pdf

# -- PDF 파일 경로 입력 및 로드.
loader = PyPDFLoader("./SPRI_AI_Brief_2024_May.pdf")

--2024-06-02 16:42:22--  https://raw.githubusercontent.com/DoranLyong/langchain-tutorial-kor/main/Agent/data/SPRI_AI_Brief_2024_May.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5012975 (4.8M) [application/octet-stream]
Saving to: ‘SPRI_AI_Brief_2024_May.pdf’

SPRI_AI_Brief_2024_ 100%[===================>]   4.78M  4.98MB/s    in 1.0s    

2024-06-02 16:42:24 (4.98 MB/s) - ‘SPRI_AI_Brief_2024_May.pdf’ saved [5012975/5012975]



In [11]:
# -- 텍스트 분할기를 사용하여 문서를 분할.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [12]:
# -- PDF 문서를 로드 및 분할.
split_docs = loader.load_and_split(text_splitter)

print(f"Num of chunks: {len(split_docs)}")
print(split_docs[1])

Num of chunks: 43
page_content='2024 년 5월호\nⅠ. 인공지능 산업 동향 브리프\n 1. 정책/법제 \n   ▹ 유엔총회 , AI의 안전한 사용을 위한 결의안을 만장일치로 채택········································· 1\n   ▹ 미국과 영국, AI 모델의 안전 테스트 협력을 위한 MOU 체결·········································· 2\n   ▹ 미국 백악관 예산관리국 , 연방정부의 AI 사용에 관한 정책 발표······································· 3\n   ▹ 캐나다 정부, AI 산업 육성에 2조 4,000 억 원 투자 계획················································· 4\n   ▹ 과기정통부 , AI전략최고위협의회 출범 및 AI 혁신성장과 AI 일상화 추진························· 5\n \n 2. 기업/산업 \n   ▹ 엔비디아 , 차세대 AI 플랫폼 ‘블랙웰 (Blackwell)’ 발표······················································· 6\n   ▹ 일론 머스크의 xAI, ‘그록-1.5’ 및 멀티모달 버전 ‘그록-1.5V’ 공개································ 7\n   ▹ 메타, 오픈소스 LLM ‘라마’의 최신 버전 ‘라마 3’ 공개····················································· 8\n   ▹ 바이두 AI 챗봇 ‘어니봇 ’의 사용자 2억 명 돌파 ································································ 9\n 3. 기술/연구\n   ▹ EU집행위원회 , 연구 활동에서 생성 AI의 책임 있는 사용에 관한 지침 발표················· 10' metadata={'source'

In [13]:
vector = FAISS.from_documents(split_docs, embedding_model)

In [14]:
# -- 벡터스토어를 검색기로 변환.
retriever = vector.as_retriever()

In [15]:
# -- PDf 문서에서 Query 에 대한 관련성 높은 Chunk 를 가져옴.
query = "엔비디아, 차세대 AI 플랫폼 ‘블랙웰(Blackwell)'에 대한 내용을 알려줘"
retriever.get_relevant_documents(query)[0]

/home/kist-cvipl/anaconda3/envs/pt113/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(page_content='n(연구기관 ) 연구 활동에서 AI의 책임 있는 사용을 지원하고 기관 내 AI 시스템 개발과 사용을 \n적극적으로 감독하며 , AI 지침을 연구 윤리와 모범관행 지침에 통합 필요\n∙연구기관은 AI 사용에 대한 교육을 제공하거나 지원해야 하며, 기관 내 AI 활용 현황을 파악하고 \n기술의 한계를 분석해 피드백을 제공해야 함\n∙자체적으로 AI 도구를 개발하거나 클라우드 기반 AI 도구를 활용해 개인정보보호와 기밀유지가 \n보장되는 환경에서 연구 데이터를 AI 도구에 입력해야 함\nn(자금 지원기관 ) 연구에서 책임 있는 AI의 사용을 장려하고 내부 절차에서 AI 사용을 검토하며 , \n빠르게 변화하는 AI 분야의 동향을 적극적으로 모니터링 필요\n∙책임있고 윤리적인 AI 활용을 지원하는 자금 지원 프로그램을 설계하고 , 평가와 심사 과정에서 AI를 \n투명하게 사용하며 , 자금 신청 준비 과정 및 연구 활동에서 AI 사용 내역의 투명한 공개를 요구\n☞ 출처 : European Commission, Guidelines on the responsible use of generative AI in research developed by the \nEuropean Research Area Forum, 2024.03.20.', metadata={'source': './SPRI_AI_Brief_2024_May.pdf', 'page': 12})

## PDF 도구 생성

In [16]:
# -- langchain 패키지의 tools 모듈에서 retriever 도구를 생성하는 함수를 가져옴.
from langchain.tools.retriever import create_retriever_tool

In [17]:
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="PDF 문서에서 정보를 검색합니다. '2024년 5월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

## 도구 목록 정의

In [19]:
# -- tools 리스트 정의.
tools = [retriever_tool] # [web_search_tool, pdf_search_tool]

# LlaMA3-8B

In [22]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFacePipeline  # 새롭게 추가

from langchain import hub
from langchain.agents import create_json_chat_agent, AgentExecutor

## Model load
* https://huggingface.co/blog/langchain, 2024. ; huggingface 모델을 langchain에서 활용하려면? [github link](https://github.com/langchain-ai/langchain/tree/master/libs/partners/huggingface)
* https://python.langchain.com/v0.1/docs/integrations/platforms/huggingface/
* https://huggingface.co/cognitivecomputations/dolphin-2.9.1-llama-3-8b ; 활용할 모델
* [Hugging Face Local Pipelines](https://python.langchain.com/v0.1/docs/integrations/llms/huggingface_pipelines/)

In [12]:
#llm = ChatOpenAI(
#    base_url="http://localhost:1234/v1",
#    api_key="lm-studio",
#    model="cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
#    temperature=0,
#)

* 위 방식은 모델을 서버에 로드하여 사용해야함.
* 하지만, 나는 local gpu에 로드해서 사용하고 싶음.
    *  이건 LM Studio라는 것을 활용하는 듯 함 (추후에 해보겠음).

In [23]:
model_ids = {"phi-3":"microsoft/Phi-3-mini-4k-instruct",
            "llama3":"cognitivecomputations/dolphin-2.9.1-llama-3-8b"}


llm = HuggingFacePipeline.from_model_id(
        model_id=model_ids["llama3"],
        task="text-generation",
        pipeline_kwargs={
                    "max_new_tokens": 100,
                    "top_k": 50,
                    "temperature": 0.1,
                    },
        )

llm.invoke("Hugging Face is")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:32<00:00,  8.05s/it]
Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


* Colab에서는 RAM이 부족해서 실패 ㅜㅜ. 
* LlaMA3-8B를 로드하려면 최소 16GB RAM, 8GB VRAM이 필요함. 

## Prompt template

In [21]:
# Use Prompt Template for JSON Agent
json_prompt = hub.pull("hwchase17/react-chat-json")

# 가져온 JSON 프롬프트를 출력합니다.
print(json_prompt)

NameError: name 'hub' is not defined

## Build Agent

In [20]:
# Agent
llama3_agent = create_json_chat_agent(llama3, tools, json_prompt)

NameError: name 'create_json_chat_agent' is not defined